# Red neuronal Fully-Connected para Clasificación binaria



##### 1. Importaciones

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import logging

##### 2. Ajuste detalles mensajes de errores

In [ ]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)

##### 3. Importación DB (IMDB)

In [ ]:
from tensorflow.keras.datasets import imdb


##### 4. Carga de datos

In [ ]:
# se divide los datos para entrenar modelo y para testear rendimiento del modelo entrenado
(train_data, train_labels),(test_data, test_labels)=imdb.load_data(num_words=10000)

##### 5. Función para contar frecuencia palabras

In [ ]:
# Devuelve la frecuencia
def to_vector(review):
    x=np.zeros(10000)
    for n in review:
        x[n]+=1.
        return x

##### 6. Conversión de datos

In [ ]:
# Conversión de datos para entrenamiento modelo.
X_train = np.zeros((25000, 10000))
for i, x in enumerate(train_data):
    X_train[i]=to_vector(x)


In [ ]:
# Conversión de datos para evalución modelo entrenamiento.
X_test =np.zeros((25000, 10000))
for i, x in enumerate(test_data):
    X_test[i]= to_vector(x)
    

##### 7. División de los datos de entrenamiento en 70% y 30%

In [ ]:
# Asignación de un índice aleatorio y selección del 70%.
rand_idx = np.random.permutation(25000)
n_train = int(0.7 * np.size(train_data, 0))

# Selección del 70% de datos para entrenamiento y del 30% para validación del modelo.
X_train, X_val = X_train[rand_idx[:n_train]], X_train[rand_idx[n_train:]]

# 30% restante para validación
y_train, y_val = train_labels[rand_idx[:n_train]], train_labels[rand_idx[n_train:]]



##### 8. Definición del modelo

In [ ]:
# Creación de variables para almacenar modelos e historiales de entrenamiento.
models ={}
his = {}

#Variable para almacenar cantidad de neuronas por capa:

Hs = [2,4,6,8]


# Definición del modelo
for h in Hs:   
    models[h] = tf.keras.Sequencial()
    models[h].add(tf.keras.layers.Dense(h, input_shape=(10000,), activation='relu'))
    models[h].add(tf.keras.layers.Dense(1,activation='sigmoid'))
    
    

##### 9. Compilación del modelo

In [ ]:
#Algotimo de Optimización, Función de coste y medidas de rendimiento
models[h].compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

##### 10. Entrenamiento del modelo

In [ ]:
# Entrenamiento y cálculo tiempo de entrenamiento
start = time.time()
history = models[h].fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_val,y_val), verbose=1)
end_time = time.time()-start

##### 11. Historial del entrenamiento

In [ ]:
# Variable con el historial del entrenamiento
his[h] = history.history

#Impresión del historial de entrenamiento de "h":
print('Entrenamiento con h= {:.0f}; Parámetros (#)= {:.0f}; Prec. Validación= {:.3f}; t. entrenamiento (s)={:.3f}'
      .format(h,models[h].count_params(),his[h]['val_acc'][-1],end_time))

##### 12. Gráfico para valoración función coste

In [ ]:
# Función coste: la discrepancia entre las probabilidades predichas por el modelo y las etiquetas verdaderas. (Train)
fig,ax = plt.subplots(1,2,figsize=(16,6))
for h in Hs:
    ax[0].plot(his[h]['loss'], label='{:.0f} neurons'.format(h))
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    
    ax[1].plot(his[h]['val_loss'], label='{:.0f} neurons'.format(h))
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Loss')

ax[0].legend()
ax[1].legend()
ax[0].grid()
ax[0].set_title('Train')
ax[1].set_title('Validation')
ax[1].grid()
fig.suptitle('Loss (Binary Crossentropy)',fontsize=20)
plt.show();

##### 13. Gráfico para valoración de la precisión del entremiento

In [ ]:
# Qué tán bien esta funcionando el entrenamiento para valorar si hay sobreajuste. (Validation)
fig,ax = plt.subplots(1,2,figsize=(16,6))
for h in Hs:
    ax[0].plot(his[h]['acc'], label='{:.0f} neurons'.format(h))
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Accuracy')
    
    ax[1].plot(his[h]['val_acc'], label='{:.0f} neurons'.format(h))
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Accuracy')

ax[0].legend()
ax[1].legend()
ax[0].grid()
ax[0].set_title('Train')
ax[1].set_title('Validation')
ax[1].grid()
fig.suptitle('Accuracy')
plt.show();